In [1]:
import numpy as np

In [16]:
def UserSim(n, parameter):
    """
    input :
    n - the number to of users to simulate
    parameter - the rate parameter (lambda)
    
    output: 
    A list of exponential random variable simulations - specifically, the prbability density for each
    simulated random variable instance.
    """
    np.random.seed(42)
    return list(np.random.exponential(1.0 / parameter, n)) #1st parameter is scale = 1/parameter given aka lambda

In [3]:
def HurdleFun(user_quit_times, breakpoints):
    '''
    user_quit_times: list of times at which user quit
    breakpoints: list of breakpoints
    '''
    user_quit_times = np.sort(user_quit_times)
    total_users = user_quit_times.size
    total_quit_prev = 0
    user_quit_bp = list()
    
    for bp in breakpoints:
        # Get the total users who quit so far
        total_quit = user_quit_times[user_quit_times < bp].size
        # Subtract the total users who quit till previous breakpoint to get users who quit at current breakpoint
        user_quit_bp.append(total_quit - total_quit_prev)
        # Keep track of users who quit so far
        total_quit_prev = total_quit
    
    #Lastly add the remaining users who didn
    remaining_users = total_users - total_quit_prev
    user_quit_bp.append(remaining_users)
    return user_quit_bp
        
HurdleFun([.20, .40], [.25, .5])

[1, 1, 0]

In [4]:
x = [.25, .45, .75]
breaks = [.5]

In [5]:
hf = HurdleFun(x, breaks)
hf

[2, 1]

In [6]:
def exp(lam, x):
    return np.log(1 - np.exp(-1*x*lam))

In [7]:
hf, breaks

([2, 1], [0.5])

In [8]:
def cdf(lam, x):
    '''
    Returns exponential distribution's cdf when lambda and x are given
    '''
    return (1 - np.exp(-1*x*lam))

def EstLam2(hurdles, breaks):
    '''
    Currying function to return another function
    Inputs:
        hurdles: output of HurdleFun
        breaks: list of breakpoints
    Returns: function instance for calcluating log_likelihood given the setup(hurdles, breaks)
    
    TODO: Convert into decorator function
    '''
    
    total_users = sum(hurdles)
    # keep track of m0, m1 and m2
    m0 = hurdles[0]
    bp1 = breaks[0]
    m2 = hurdles[-1]
    bp_last = breaks[-1]
    m1 = total_users - m0 - m2
    
    def log_likehood(lam):
        """
        Specialized function to be called as a lambda, which takes the lam list and
        returns the log_likelihood
        
        """
        log_like = (m0 * np.log(cdf(lam, bp1))) + (m2 * -1*lam*bp_last)
        # If there are users in m1, then add relevant sums to log likelihood
        if m1 != 0:
            for i in range(len(breaks) - 1):
                log_like += hurdles[i+1]*np.log(cdf(lam, breaks[i + 1]) - cdf(lam, breaks[i]))
        return log_like
    
    return log_likehood

In [9]:
p = EstLam2(hf, breaks)

In [10]:
p(1)

-2.3655042591343771

### Part 4

In [11]:
def MaxMLE(survival_list, breakpoints, lambda_list):
    """
    Given the list of survival of users in the form of the output of hurdlefun, breakpoints list and 
    the possible values of lambda, outputs the best lambda for which the MLE estimates are lowest
    Does that by using the EstLam2 function to get the MLE estimate
    
    Input: Survival list of users [], breakpoints [], possible lambda values []
    Output: best lambda float
    """
    PRT = EstLam2(survival_list, breakpoints)
    mle_list = [PRT(x) for x in lambda_list]
    index = np.argmax(mle_list)
    
    return lambda_list[index]

In [12]:
print(MaxMLE( HurdleFun(x, breaks), breaks, list(np.arange(.1, 3, .05))))

2.2


##### Part 4a.

In [17]:
np.random.seed(42)
# Function to estimate lambda using MLE appraoch
def EstLam1(quitting_time): return 1.0/np.mean(np.array(quitting_time))


# Calcualate the difference in the estimates of lambda by EstLam1 and EstLam2
for breaks in [[.25,.75],[.25,3],[.25,10]]:
    lambda_diff = []
    lambda1=[]
    lambda2=[]
    for i in range(0,1000):
        samples = UserSim(100, 1)
        lmbda1 = EstLam1(samples)
        lmdba2 = MaxMLE(HurdleFun(samples, breaks), breaks, list(np.arange(.1, 3, .05)))
        lambda1.append(lmbda1)
        lambda2.append(lmdba2)
        diff=lmbda1-lmdba2
        lambda_diff.append(diff)
    print('For break:{}'.format(breaks))
    print('Average lambda1: {} and lambda2: {}'.format(np.round(np.mean(lambda1),4), np.round(np.mean(lambda2),4)))
    print('Average difference is {}'.format( np.round(np.mean(lambda_diff),4)))

For break:[0.25, 0.75]
Average lambda1: 1.0932 and lambda2: 1.15
Average difference is -0.0568
For break:[0.25, 3]
Average lambda1: 1.0932 and lambda2: 1.1
Average difference is -0.0068
For break:[0.25, 10]
Average lambda1: 1.0932 and lambda2: 1.35
Average difference is -0.2568


Moving the breakpoints doesn't effect the estimate of lambda. 

#### Part 4b.

As we saw that changing the breakpoints doesn't effect the estimate of lambda, we can design the breakpoints with the interval which satisfy our business requirements without being limited by the intervals. Since the exponential distribution is memoryless, changing the interval don't effect much